In [ ]:
import sys
import os 
import warnings
import numpy as np

pkg_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(pkg_path)

np.set_printoptions(threshold=np.inf, suppress=True, linewidth=np.inf)
warnings.filterwarnings(action='ignore', category=UserWarning)

import shtmbss2.addsrc
from shtmbss2.common.config import *
from shtmbss2.core.logging import log

In [ ]:
RuntimeConfig.backend = Backends.BRAIN_SCALES_2
RuntimeConfig.plasticity_location = PlasticityLocation.OFF_CHIP 

In [ ]:
shtm = None
if RuntimeConfig.backend == Backends.BRAIN_SCALES_2:
    import shtmbss2.brainscales2.patches
    from shtmbss2.brainscales2.hardware import hardware_initialization
    from shtmbss2.brainscales2.network import SHTMTotal
    # TODO: remove once grenade supports dense inter-population-view projections
    neuronPermutation = []
    shtm = SHTMTotal(use_on_chip_plasticity=RuntimeConfig.plasticity_location == PlasticityLocation.ON_CHIP)
    alphabet_size = shtm.p.Network.num_symbols
    num_neurons_per_symbol = shtm.p.Network.num_neurons
    for a in range(alphabet_size):
        # dendrites
        for i in range(num_neurons_per_symbol):
            neuronPermutation.append((a * num_neurons_per_symbol + i) * 2)
    for a in range(alphabet_size):
        # somas
        for i in range(num_neurons_per_symbol):
            neuronPermutation.append((a * num_neurons_per_symbol + i) * 2 + 1)
    for i in range(alphabet_size * num_neurons_per_symbol * 2, 512):
        neuronPermutation.append(i)

    hardware_initialization(neuronPermutation=neuronPermutation)
elif RuntimeConfig.backend == Backends.NEST:
    from shtmbss2.nest.network import SHTMTotal
    
from shtmbss2.common.network import NeuronType, RecTypes
from shtmbss2.common.optimization import GridSearch 

## Config

In [ ]:
log.handlers[LogHandler.FILE].setLevel(logging.ESSENS)

In [ ]:
experiment_type = ExperimentType.OPT_GRID
model_type = SHTMTotal
experiment_id = "weights-cons"
experiment_num = 1

## Run

In [ ]:
opt = GridSearch(experiment_type=ExperimentType.OPT_GRID, 
                 model_type=SHTMTotal, 
                 experiment_id="weights-cons", 
                 # experiment_num=1
                 )

opt.run(steps=50)